In [8]:
# Import soft alignment functions

import os
from Bio import SeqIO
import torch
from soft_align import get_data_matrix, get_matches, get_longest_path



In [9]:
# Specify the directory paths where sequence embeddings and fasta sequences are stored
embedding_directory = './soft_align_example_data/example_embeddings/'

fasta_directory = './soft_align_example_data/'
fasta_file = 'example.fasta'


In [10]:
# Load fasta file as a dictionary
# Keys in seqs dictionary are sequence names
# Values in seqs dictionary are the information for each sequence stored in the fasta file

seqs = SeqIO.to_dict(SeqIO.parse(os.path.join(fasta_directory, fasta_file), 'fasta'))

# Create a list of sequence names
seq_names = list(seqs.keys())
print(seq_names)

['YP_006990334.1', 'YP_001468397.1', 'WP_016056174.1']


In [11]:
# Print sequence from seqs dictionary

seqs['YP_006990334.1'].seq[0:40]

Seq('MQNNKNFQNVLLAHINNIKDLPLKARIDYFEDDKDDLVIN')

In [12]:
# Create a string of the sequences being compared

seq_1_str =  str(seqs[seq_names[0]].seq)
seq_2_str =  str(seqs[seq_names[1]].seq)


In [13]:
# Print the length of the sequence strings

print("Length Sequence 1: ", len(seq_1_str))
print("Length Sequence 2: ", len(seq_2_str))

Length Sequence 1:  135
Length Sequence 2:  135


In [14]:
# Create dictionary containing the embeddings of sequences
# Sequence name and representation are stored in the dictionary

seq_1_embedding = torch.load(f"{embedding_directory + seq_names[0]}.pt")
seq_2_embedding = torch.load(f"{embedding_directory + seq_names[1]}.pt")

In [15]:
# Embeddings created by ESM2 are saved in a dictionary with the following structure. 
# When utilizing different formats, it's important to correctly use the "representations" field 
# that corresponds to the model being used.

print("Sequence 1 Dictionary Keys: ", seq_1_embedding.keys())
print("Sequence 2 Dictionary Keys: ", seq_2_embedding.keys())

Sequence 1 Dictionary Keys:  dict_keys(['label', 'representations'])
Sequence 2 Dictionary Keys:  dict_keys(['label', 'representations'])


In [16]:
# Generate a matrix containing cosine distances of amino acid embeddings from sequence representations

data = get_data_matrix(seq_1_embedding,seq_2_embedding)
data.iloc[0:5, 0:5]

,0,1,2,3,4
0,0.947351,0.838340,0.865979,0.830881,0.826368
1,0.866956,0.878017,0.888858,0.862527,0.836949
2,0.886032,0.876636,0.897953,0.880396,0.876158
3,0.852143,0.892888,0.901820,0.852618,0.876243
4,0.843087,0.852053,0.927681,0.866988,0.867131


In [17]:
# Print the shape of the cosine distance matrix
# The shape corresponds to (length of sequence 1, length of sequence 2)

data.shape

(135, 135)

In [18]:
# Generate matches of amino acids from the cosine distance matrix

matches = get_matches(seq_1_str, seq_2_str, data)
matches[0:5]

[(25, 23), (59, 55), (107, 104), (133, 132), (126, 126)]

In [19]:
# Generate a longest path of matches that diagonally traverse the matrix, to be considered a soft alignment
longest_path = get_longest_path(data, matches)
longest_path [0:5]

[(0, 0), (3, 1), (4, 2), (5, 3), (6, 4)]